In [95]:
import pandas as pd
import numpy as np
from datetime import datetime
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from slugify import slugify
import pycountry

In [96]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [97]:
df = pd.read_csv('csv/localities.csv', encoding='utf-8')
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [98]:
newDfC = pd.DataFrame(data = df['Continent'].unique())

In [105]:
columns = ['id', 'createdDate', 'updatedDate', 'slug', 'latitude', 'longitude', 'internalNote', \
          'isPopular', 'order', 'statusId', 'canShowAds', 'isActive', \
          'imageId', 'coverImageId', 'type', 'countryContinentId', 'regionCountryId', \
          'regionContinentId', 'code']

In [106]:
dfCon = newDfC

In [107]:
dfCon = pd.concat([dfCon, pd.DataFrame(columns = columns)])

In [108]:
dfCon.rename(columns = {0:'name'}, inplace = True)

In [109]:
def get_photo_name(value):
    try:
        return df[:][df['Continent'] == str(value)]['Photo name'].values[0]
    except:
        return ''

In [110]:
def get_country_name(value):
    try:
        name = pycountry.countries.get(name=str(value))
        return name.alpha_2
    except:
        return ''

In [111]:
dfCon['id'] = dfCon.index + 1
dfCon['order'] = ''
dfCon['createdDate'] = dt_string
dfCon['updatedDate'] = dt_string
dfCon['internalNote'] = ''
dfCon['isPopular'] = False
dfCon['statusId'] = 7
dfCon['canShowAds'] = False
dfCon['isActive'] = True
dfCon['type'] = 'Continent'
dfCon['photoName'] = dfCon['name'].apply(get_photo_name)

In [112]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="trynego")

In [113]:
def get_lat(value):
    try:
        return geolocator.geocode(str(value))[1][0]
    except:
        return None

In [114]:
def get_lot(value):
    try:
        return geolocator.geocode(str(value))[1][1]
    except:
        return None

In [115]:
def set_slug_id():
    try:
        for index, row in dfCon.iterrows():
            dfCon.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [116]:
dfCon['latitude'] = dfCon['name'].apply(get_lat)
dfCon['longitude'] = dfCon['name'].apply(get_lot)

In [117]:
dfCon = dfCon[:][dfCon['name'].notnull()]

In [118]:
newDfCountries = pd.DataFrame(data = df['Country'].unique())

In [119]:
def get_photo_name_c(value):
    try:
        return df[:][(df['Country'] == str(value)) & (df['Region'] == '-')]['Photo name'].values[0]
    except:
        return ''

In [120]:
dfCountry = pd.concat([newDfCountries, pd.DataFrame(columns = columns)])

In [121]:
dfCountry.rename(columns = {0:'name'}, inplace = True)

In [122]:
def set_slug_id():
    try:
        for index, row in dfCountry.iterrows():
            dfCountry.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [123]:
dfCountry['id'] = dfCountry.index + dfCon.shape[0]+1
dfCountry['order'] = ''
dfCountry['createdDate'] = dt_string
dfCountry['updatedDate'] = dt_string
dfCountry['internalNote'] = ''
dfCountry['isPopular'] = False
dfCountry['statusId'] = 7
dfCountry['canShowAds'] = False
dfCountry['isActive'] = True
dfCountry['type'] = 'Country'
dfCountry['photoName'] = dfCountry['name'].apply(get_photo_name_c)
dfCountry['code'] = dfCountry['name'].apply(get_country_name)

In [124]:
dfCountry['latitude'] = dfCountry['name'].apply(get_lat)
dfCountry['longitude'] = dfCountry['name'].apply(get_lot)

In [125]:
def set_continent_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Country'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [126]:
def set_image_id_c(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

In [127]:
dfCountry['countryContinentId'] = dfCountry['name'].apply(set_continent_id)

In [128]:
dfCountry['coverImageId'] = dfCountry['name'].apply(set_image_id_c)

In [129]:
dfCountry = dfCountry[:][dfCountry['name'].notnull()]

In [130]:
newDfR = pd.DataFrame(data = df[["Region"]])

In [131]:
dfReg = pd.concat([newDfR, pd.DataFrame(columns = columns)])

In [132]:
dfReg.rename(columns = {'Region':'name'}, inplace = True)

In [133]:
def set_slug_id():
    try:
        for index, row in dfReg.iterrows():
            dfReg.loc[index, ['slug']] = [slugify(row['name'])]
    except:
        pass
        
set_slug_id()

In [134]:
def get_photo_name_r(value):
    try:
        return df[:][df['Region'] == str(value)]['Photo name'].values[0]
    except:
        return ''

In [135]:
temp = (dfCon.shape[0]+1) + (dfCountry.shape[0]+1)
dfReg['id'] = dfReg.index + temp 
dfReg['order'] = ''
dfReg['createdDate'] = dt_string
dfReg['updatedDate'] = dt_string
dfReg['internalNote'] = ''
dfReg['isPopular'] = False
dfReg['statusId'] = 7
dfReg['canShowAds'] = False
dfReg['isActive'] = True
dfReg['type'] = 'Region'
dfReg['photoName'] = dfReg['name'].apply(get_photo_name_r)
dfReg['code'] = dfReg['name'].apply(get_country_name)

In [136]:
dfReg['latitude'] = dfReg['name'].apply(get_lat)
dfReg['longitude'] = dfReg['name'].apply(get_lot)

In [137]:
def set_continent_reg_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Region'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [138]:
def set_region_reg_id(value):
    try:
        return dfCountry[:][dfCountry['name'] == df[:][df['Region'] == str(value)]['Country'].values[0]]['id'].values[0]
    except:
        return ''

In [139]:
dfReg['regionContinentId'] = dfReg['name'].apply(set_continent_reg_id)

In [140]:
dfReg['regionCountryId'] = dfReg['name'].apply(set_region_reg_id)

In [141]:
dfReg = dfReg[:][dfReg['name'].notnull()]

In [142]:
def set_image_id(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

dfReg['coverImageId'] = dfReg['photoName'].apply(set_image_id)

In [143]:
def set_image_id_c(value):
    try:
        return image[:][image['name'].str.lower() == 'region_' + str(value).lower().replace(" ", "_")]['id'].values[0]
    except:
        return ''

dfCountry['coverImageId'] = dfCountry['photoName'].apply(set_image_id_c)

In [144]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [145]:
newDf = pd.concat([dfCon, dfCountry, dfReg])

In [146]:
newDf = newDf[:][(newDf['name'] != '') & (newDf['name'] != '-') & (newDf['name'].notnull())]

In [147]:
newDf['id'] = [i for i in range(1, newDf.shape[0] + 1)]

In [148]:
newDf['order'] = newDf.index + 1

In [149]:
newDf['latitude'] = newDf['latitude'].fillna(1)
newDf['longitude'] = newDf['longitude'].fillna(1)

In [150]:
newDf.to_csv('out/localities.csv', index=False, encoding='utf-8')

In [151]:
newDf.shape

(234, 21)

In [152]:
newDfCopy = newDf

In [153]:
def remove_number(value):
    return ''.join([i for i in str(value) if not i.isdigit()])

In [154]:
newDfCopy['name'] = newDfCopy['name'].apply(remove_number)

In [164]:
newDfCopy = newDfCopy.drop_duplicates(subset='name', keep="first")

In [165]:
newDfCopy.to_csv('out/localities_main.csv', index=False, encoding='utf-8')

In [166]:
newDfCopy[:][newDfCopy['coverImageId'] == 1474]

,name,id,createdDate,updatedDate,slug,latitude,longitude,internalNote,isPopular,order,statusId,canShowAds,isActive,imageId,coverImageId,type,countryContinentId,regionCountryId,regionContinentId,code,photoName
78,Foz do Iguacu,108,2023-06-26 14:46:23,2023-06-26 14:46:23,foz-do-iguacu,-25.540148,-54.585814,,False,79,7,False,True,NaN,1474,Region,NaN,21,3,,16_2_Trynego_Brazil_Foz do Iguacu
